# Bronze to Silver: Data Cleaning and Transformations for Dimension Tables

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, IntegerType, FloatType, TimestampType, DateType

catalog_name = "ecommerce"

# Brands

In [0]:
df_bronze =spark.table(f"{catalog_name}.bronze.brz_brands")
df_bronze.show(10)


In [0]:
df_silver = df_bronze.withColumn("brand_name", F.trim(F.col("brand_name")))
df_silver.show(10)

In [0]:
df_silver = df_silver.withColumn("brand_code",F.regexp_replace(F.col("brand_code"), r'[^A-Za-z0-9]',''))
df_silver.show(10)

In [0]:
df_silver.select("category_code").distinct().show()


In [0]:
anomalies = {
    "GROCERY": "GRCY",
    "BOOKS" : "BKS",
    "TOYS" : "TOY"
}
df_silver = df_silver.replace(to_replace=anomalies,subset=["category_code"])
df_silver.select("category_code").distinct().show()

In [0]:
# write raw data into silver layer -> ecommerce.silver.slv_brands
df_silver.write.format("delta") \
    .mode("overwrite") \
    .option("mergeScehma", "true") \
    .saveAsTable(f"{catalog_name}.silver.slv_brands")


# Category


In [0]:
df_bronze = spark.table(f"{catalog_name}.bronze.brz_category")
df_bronze.show(10)

In [0]:
df_duplicates = df_bronze.groupBy("category_code").count().filter("count > 1")
display(df_duplicates)

In [0]:
df_silver = df_bronze.dropDuplicates(["category_code"])
df_silver.show(10)


In [0]:
df_silver = df_silver.withColumn("category_code",F.upper(F.col("category_code")))
display(df_silver)

In [0]:
# write raw data into silver layer -> ecommerce.silver.slv_category
df_silver.write.format("delta")\
  .mode("overwrite")\
  .option("mergeScehma", "true")\
  .saveAsTable(f"{catalog_name}.silver.slv_category")

# Products

In [0]:
# read the raw data from bronze layer
df_bronze = spark.table(f"{catalog_name}.bronze.brz_products")

# get row and column count
row_count, column_count = df_bronze.count(), len(df_bronze.columns) 

print(f"Row_Count: {row_count}")
print(f"Column_Count : {column_count}")

In [0]:
display(df_bronze.limit(5))

In [0]:
# cleaning the weight_grams columns (remove the g)
df_bronze.select("weight_grams").show(5, truncate=False)

In [0]:
df_silver = df_bronze.withColumn(
    "weight_grams", 
    F.regexp_replace(F.col("weight_grams"), "g", "").cast(IntegerType())
)
df_silver.select("weight_grams").show(5, truncate=False)

In [0]:
# Check Length (comma instead of dot)
df_silver.select("length_cm").show(3,truncate=False)

In [0]:
df_silver = df_silver.withColumn(
    "length_cm", 
    F.regexp_replace(F.col("length_cm"), ",", ".").cast(FloatType())
)
df_silver.select("length_cm").show(3)

In [0]:
# category_Code and brand_code are in lower case, convert to upper case
df_silver.select("category_Code", "brand_code").show(3, truncate=False)


In [0]:
df_silver = df_silver.withColumn(
    "category_code",
    F.upper(F.col("category_Code"))
).withColumn(
    "brand_code",
    F.upper(F.col("brand_code"))
)
df_silver.select("category_code", "brand_code").show(3)

In [0]:
# Spelling mistakes in "material" column
df_silver.select("material").distinct().show()

In [0]:
df_silver = df_silver.withColumn(
    "material",
    F.when(F.col("material") == "Coton", "Cotton")
    .when(F.col("material") == "Ruber", "Rubber")
    .when(F.col("material") == "Alumium", "Aluminum")
    .otherwise(F.col("material"))
)
df_silver.select("material").distinct().show()

In [0]:
# Negative values in rating_count
df_silver.filter(F.col("rating_count")<0).select("rating_count").show(3)

In [0]:
df_silver = df_silver.withColumn(
    "rating_count",
    F.when(F.col("rating_count").isNotNull(), F.abs(F.col("rating_count")))
     .otherwise(F.lit(0)) # if null, replace with 0
)

In [0]:
# checking the final cleaned data
df_silver.select(
    "weight_grams",
    "length_cm",
    "category_code",
    "brand_code",
    "rating_count"
).show(10, truncate=False)

In [0]:
# write raw data into silver layer -> ecommerce.silver.slv_products
df_silver.write.format("delta")\
        .mode("overwrite")\
        .option("mergeSchema", "true")\
        .saveAsTable(f"{catalog_name}.silver.slv_products")

# Customers

In [0]:
# read the raw data from bronze table of customers
df_bronze = spark.table(f"{catalog_name}.bronze.brz_customers")

# get row and columns count
row_count, column_count = df_bronze.count(), len(df_bronze.columns)
print(f"Row_count: {row_count}")
print(f"Column_count: {column_count}")
display(df_bronze.limit(10))

In [0]:
null_count = df_bronze.filter(F.col("customer_id").isNull()).count()
null_count

In [0]:
df_bronze.filter(F.col("customer_id").isNull()).show(3)  # over 300 NULL values in customerid column

In [0]:
# Drop the rows where customerid is null
df_silver = df_bronze.dropna(subset=["customer_id"])

# get row_count 
row_count = df_silver.count()
print(f"Rows count after droping null values: {row_count}")

In [0]:
# Handle null values in Phone Number
null_count = df_silver.filter(F.col("phone").isNull()).count() 
print(f"No.of nulls in phone: {null_count}")

df_silver.filter(F.col("phone").isNull()).show(3)

In [0]:
# Fill null values in Phone with 'Not Available'
df_silver = df_silver.fillna("Not Available", subset=["phone"])

# Sanity check (if any NULL still exists)
df_silver.filter(F.col("phone").isNull()).show()

In [0]:
# write raw data into silver layer -> ecommmerce.silver.slv_customers
df_silver.write.format("delta")\
        .mode("overwrite")\
        .option("mergeSchema", "true")\
        .saveAsTable(f"{catalog_name}.silver.slv_customers")

# Date -> Calendar(silver_layer)

In [0]:
# read the raw data form bronze layer
df_bronze = spark.table(f"{catalog_name}.bronze.brz_date")
# get row and columns count
row_count, column_count = df_bronze.count(), len(df_bronze.columns)
print(f"Row count: {row_count}, Column count: {column_count}")
display(df_bronze.limit(10))

In [0]:
df_bronze.printSchema()

In [0]:
# Converting string date column into date type
df_silver = df_bronze.withColumn("date", F.to_date(df_bronze["date"], "dd-MM-yyyy"))

In [0]:
print(df_silver.printSchema())
df_silver.show(5)

In [0]:
# FInd the duplicates
duplicates = df_silver.groupBy("date").count().filter("count>1")

print(f"Total duplicates: {duplicates}")
display(duplicates)

In [0]:
# Remove the duplicates
df_silver  = df_silver.dropDuplicates(["date"])

row_count = df_silver.count()
print(f"Rows count after droping duplicates: {row_count}")

In [0]:
# Normalize the day_name
df_silver = df_silver.withColumn("day_name", F.initcap(F.col("day_name")))
df_silver.show(5)

In [0]:
# Convert negstive week_of_year to positive
df_silver = df_silver.withColumn("week_of_year", F.abs(F.col("week_of_year")))
df_silver.show(3)

In [0]:
# ENhance week_of_year and quater column
df_silver = df_silver.withColumn("quarter", F.concat_ws("", F.concat(F.lit("Q"),F.col("quarter"),F.lit("-"),F.col("year"))))

df_silver = df_silver.withColumn("week_of_year",F.concat_ws("-", F.concat(F.lit("Week"), F.col("week_of_year"), F.lit("-"), F.col("year"))))

df_silver.show(3)

In [0]:
# Rename the week_of_year column
df_silver = df_silver.withColumnRenamed("week_of_year", "week")
# display(df_silver.limit(5))

In [0]:
# write raw data into silver layer -> ecommerce.silver.slv_calendar
df_silver.write.format("delta")\
    .mode("overwrite")\
    .option("mergeSchema", "true")\
    .saveAsTable(f"{catalog_name}.silver.slv_calendar")